# Using Decision Tree

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
Data = pd.read_csv("kolkata_hrdata.csv")
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19503 entries, 0 to 19502
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        19503 non-null  object 
 1   Datetime    19503 non-null  object 
 2   PM2.5       18061 non-null  float64
 3   PM10        18071 non-null  float64
 4   NO          18738 non-null  float64
 5   NO2         18795 non-null  float64
 6   NOx         19386 non-null  float64
 7   NH3         19427 non-null  float64
 8   CO          19480 non-null  float64
 9   SO2         17907 non-null  float64
 10  O3          18161 non-null  float64
 11  Benzene     19444 non-null  float64
 12  Toluene     19438 non-null  float64
 13  Xylene      9352 non-null   float64
 14  AQI         18066 non-null  float64
 15  AQI_Bucket  18066 non-null  object 
dtypes: float64(13), object(3)
memory usage: 2.4+ MB


In [3]:
feature = []
#target = []

In [12]:
feature = Data
feature = feature.drop('City',axis=1)
feature = feature.drop('NO',axis=1)
feature = feature.drop('NO2',axis=1)
feature = feature.drop('NOx',axis=1)
feature = feature.drop('NH3',axis=1)
feature = feature.drop('SO2',axis=1)
feature = feature.drop('O3',axis=1)
feature = feature.drop('Toluene',axis=1)
feature = feature.drop('Benzene',axis=1)
feature = feature.drop('Xylene',axis=1)
feature = feature.drop('AQI_Bucket',axis=1)
feature.info()
feature.head(20)#Features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19503 entries, 0 to 19502
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  19503 non-null  object 
 1   PM2.5     18061 non-null  float64
 2   PM10      18071 non-null  float64
 3   CO        19480 non-null  float64
 4   AQI       18066 non-null  float64
dtypes: float64(4), object(1)
memory usage: 762.0+ KB


,Datetime,PM2.5,PM10,CO,AQI
0,2018-04-10 10:00:00,34.82,54.45,1.13,NaN
1,2018-04-10 11:00:00,31.82,57.80,1.02,NaN
2,2018-04-10 12:00:00,29.61,51.04,0.94,NaN
3,2018-04-10 13:00:00,33.37,51.61,0.91,NaN
4,2018-04-10 14:00:00,36.71,58.00,0.91,NaN
5,2018-04-10 15:00:00,35.88,56.18,0.86,NaN
6,2018-04-10 16:00:00,37.16,53.08,0.97,NaN
7,2018-04-10 17:00:00,37.37,53.86,0.89,NaN
8,2018-04-10 18:00:00,35.42,53.62,0.79,NaN
9,2018-04-10 19:00:00,36.37,57.87,0.99,NaN


In [13]:
df = pd.DataFrame(feature)
df.to_csv('myfile.csv')
#X = feature[:,:-1]
#Y = feature[:,-1]#y=feature[]
#print(Y)

## Node Class

In [6]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

## Tree Class

In [7]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [8]:
X = feature.iloc[:, :-1].values
Y = feature.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [9]:
regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3)
regressor.fit(X_train,Y_train)
regressor.print_tree()

KeyError: 'var_red'

In [ ]:

.
